In [4]:
import scrapy
import re
import json
import logging
import pandas as pd
import selenium
import time
import sys
import linecache
import selenium.webdriver as webdriver
import selenium.webdriver.support.ui as ui
import pymongo
import datetime
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
from scrapy.crawler import CrawlerProcess
from time import sleep
from selenium.webdriver.common.keys import Keys
from time import sleep 
from pymongo import MongoClient 

# BATA Scraping

In [16]:
# Print exception
def PrintException():
    exc_type, exc_obj, tb = sys.exc_info()
    f = tb.tb_frame
    lineno = tb.tb_lineno
    filename = f.f_code.co_filename
    linecache.checkcache(filename)
    line = linecache.getline(filename, lineno, f.f_globals)
    print ('EXCEPTION IN ({}, LINE {} "{}"): {}'.format(filename, lineno, line.strip(), exc_obj))

In [26]:
def scrape_detail_page(driver,category,collection):
    # scraping page
    result = pd.DataFrame()
    item = driver.find_elements_by_css_selector('.item.product.product-item')
    for x in range(len(item)):
        name = item[x].find_elements_by_css_selector('.product-item-link')[0].get_attribute('title')
        image = item[x].find_elements_by_css_selector('.product-image-photo')[0].get_attribute('src') 
        price = item[x].find_elements_by_css_selector('.price-wrapper')[0].get_attribute('data-price-amount') 
        sku = re.sub("product-id-","",item[x].find_elements_by_css_selector('.price-box.price-final_price')[0].get_attribute('data-price-box')) 

        # Return result
        date = datetime.datetime.now().strftime("%Y-%m-%d")
        new_result = pd.DataFrame([{'sku':sku,'name':name, 'origin_price':price, 'discounted_price':price, 'image':image,'category':category,'scraping_date':date}])
        result = result.append(new_result)
        
    # save to mongodb
    try:
        collection.insert_many(result.to_dict('records'))
    except:
        pass
    return result


In [22]:
def scrape_url(url,category,collection):
        
    # Connect to mongodb
    try: 
        conn = MongoClient() 
        print("Connected successfully!!!") 
    except:   
        print("Could not connect to MongoDB") 
    # database
    db = conn.bata
    # Created or Switched to collection names: pakaian_wanita 
    collection = db[collection]
    
    # Load WebDriver and navigate to the page url.
    # This will open a browser window.
    driver = webdriver.Firefox()
    driver.get(url)

    # Start Scraping
    result = pd.DataFrame()
    scrape = True
    page = 1
    while(scrape):
        # Go to catalogue page
        result = result.append(scrape_detail_page(driver,category,collection))
        # Go to the next page
        try:
            check_next = driver.find_elements_by_xpath('/html/body/div[1]/div[2]/div/div/main/div/div[1]/div[6]/div[2]/ul/li[3]/a')[0]
            page = page+1
            next_page = url + "&p=" + str(page)
            print("Go to next page: " + next_page)
            driver.close()
            driver = webdriver.Firefox()
            driver.get(next_page)
        except:
            PrintException()
            scrape = False
    
    driver.close()
    return result


In [29]:
# MAIN
bata_category = pd.read_csv('C:\\Users\\Annisa\\Stream\\Product Development - Analysis - QBR Jul19\\Data\\scraping\\bata_category.csv',sep=";")
result = pd.DataFrame()
for x in range(1,len(bata_category)):
    category = bata_category.iloc[x]
    print(category['category'])
    url = category['url']+"?product_list_limit=30"
    result = scrape_url(url,category['category'],category['collection'])
    print(result.head())

sports
Connected successfully!!!
EXCEPTION IN (<ipython-input-22-50710d3308d1>, LINE 28 "check_next = driver.find_elements_by_xpath('/html/body/div[1]/div[2]/div/div/main/div/div[1]/div[6]/div[2]/ul/li[3]/a')[0]"): list index out of range
Empty DataFrame
Columns: []
Index: []
sandal
Connected successfully!!!
Go to next page: https://www.bata.id/catalog/category/view/s/sandal/id/301/?product_list_limit=30&p=2
EXCEPTION IN (<ipython-input-22-50710d3308d1>, LINE 28 "check_next = driver.find_elements_by_xpath('/html/body/div[1]/div[2]/div/div/main/div/div[1]/div[6]/div[2]/ul/li[3]/a')[0]"): list index out of range
  category discounted_price  \
0   sandal           349900   
0   sandal           199900   
0   sandal           199900   
0   sandal           199900   
0   sandal           199900   

                                               image  \
0  https://www.bata.id/media/catalog/product/cach...   
0  https://www.bata.id/media/catalog/product/cach...   
0  https://www.bata.id/medi